# TP1 - Parte 3 - Comunicación y Concurrencia

# **Java**
Este ejercicio resuelve el siguiente problema:

*Problema del Zorro, el Pollo y el Maíz*  
Un granjero debe transportar un zorro, un pollo y una bolsa de maíz al otro lado de un río. El granjero tiene un bote pequeño y solo puede transportar un elemento a la vez. Sin embargo, si el zorro y el pollo se quedan solos, el zorro se come al pollo, de igual manera si el pollo queda solo con el maíz, se lo comerá.

Utilizando concurrencia se simulan las situaciones posibles de dicho
problema y su desenlace en base una entrada provista por el usuario.

## **Ejercicio sin terminar**

In [1]:
%%writefile ProblemaDelGranjero.java
import java.util.HashMap;
import java.util.Map;
import java.util.Scanner;
import java.util.concurrent.Semaphore;

public class ProblemaDelGranjero
{
	static final char LADO_A = 'A';
	static final char LADO_B = 'B';
	static final String GRANJERO = "GRANJERO";
	static final String ZORRO = "ZORRO";
	static final String POLLO = "POLLO";
	static final String MAIZ = "MAIZ";
	static final String SOLO = "SOLO";


	static Map<String, Character> personajes = new HashMap<>();
	static int transporte = 0;
	static Object lock = new Object();
	static String personajeATransportar;
	static boolean success = true;
	static boolean end = false;
	static Semaphore check = new Semaphore(0);

	public static class LadoHilo extends Thread
	{
		private char lado;

		public LadoHilo (char lado)
		{
			this.lado = lado;
		}

		public void run()
		{
			while(true)
			{
				String mensaje = "";
				synchronized (lock)
				{
					try
					{
						lock.wait();
					}
					catch(Exception e)
					{
						e.printStackTrace();
					}
					if(!success)
					{
						check.release();
						return;
					}
				}
				if(personajes.get(ZORRO) == personajes.get(POLLO) && personajes.get(GRANJERO) != personajes.get(ZORRO)
						&& personajes.get(ZORRO) == this.lado)
				{
					success = false;
					mensaje = "-El Zorro se come al pollo en el lado " + this.lado;
				}
				else if(personajes.get(POLLO) == personajes.get(MAIZ) && personajes.get(GRANJERO) != personajes.get(POLLO)
						&& personajes.get(POLLO) == this.lado)
				{
					success = false;
					mensaje = "-El Pollo se come al Maiz en el lado " + this.lado;
				}
				else
				{
					int cant = 0;
					for (String pj : personajes.keySet())
					{
						if(personajes.get(pj) == this.lado)
						{
							cant++;
							mensaje = mensaje.concat("-El " + pj + " se encuentra en el lado " + this.lado + "\n");
						}
					}
					if(cant == 4 && this.lado == LADO_B)
					{
						end = true;
						mensaje = mensaje.concat("-Todos se encuentran en el lado B");
					}
				}
				synchronized (lock)
				{
					System.out.println("--LADO " + lado + "--");
					System.out.println(mensaje);
				}
				check.release();
			}
		}
	}


	public static void main(String[] args) throws InterruptedException
	{
		Scanner entrada = new Scanner(System.in);
		LadoHilo ladoA = new LadoHilo(LADO_A);
		LadoHilo ladoB = new LadoHilo(LADO_B);
		ladoA.setDaemon(true);
		ladoB.setDaemon(true);

		personajes.put(GRANJERO, LADO_A);
		personajes.put(ZORRO, LADO_A);
		personajes.put(POLLO, LADO_A);
		personajes.put(MAIZ, LADO_A);

		ladoA.start();
		ladoB.start();

		System.out.println("Granjero, zorro, pollo y maiz se encuentran en el lado A.");
		while(true)
		{
			System.out.println(">El granjero cruza con: ");
			personajeATransportar = entrada.nextLine(); // debe ser zorro, pollo, maiz o solo
			personajeATransportar = personajeATransportar.toUpperCase();
			while (!entradaValida())
			{
				System.out.println("Error al ingresar personaje, las opciones son:");
				System.out.println("SOLO - ZORRO - POLLO - MAIZ\n");
				System.out.println(">El granjero cruza con: ");
				personajeATransportar = entrada.nextLine();
				personajeATransportar = personajeATransportar.toUpperCase();
			}
			if(transportar())
			{
				System.out.println("-Fin del programa");
				entrada.close();
				return;
			}
			System.out.println();
		}
	}


	public static boolean transportar() throws InterruptedException
	{
		if(personajes.get(personajeATransportar) == personajes.get("GRANJERO"))
		{
			char nuevoLado = (personajes.get(personajeATransportar) == LADO_A) ? LADO_B : LADO_A;
			if (personajeATransportar.equals(GRANJERO))
				System.out.println("El granjero cruza solo al lado " + nuevoLado);
			else
				System.out.println("El granjero transporta al " + personajeATransportar + " al lado " + nuevoLado);
			personajes.put(personajeATransportar, nuevoLado);
			personajes.put(GRANJERO, nuevoLado);
			synchronized (lock)
			{
				lock.notifyAll();
			}
			check.acquire(2);
			if(!success || end)
			{
				return true;
			}
		}
		else
			System.out.println("El "+ personajeATransportar + " no se encuentra del mismo lado que el granjero.");
		return false;
	}

	public static boolean entradaValida()
	{
		if(personajeATransportar.equals(MAIZ) || personajeATransportar.equals(POLLO)
		   || personajeATransportar.equals(ZORRO) || personajeATransportar.equals(SOLO))
			{
				if(personajeATransportar.equals(SOLO))
					personajeATransportar = GRANJERO;
				return true;
			}
		return false;
	}

}

Writing ProblemaDelGranjero.java


In [2]:
!javac ProblemaDelGranjero.java

In [1]:
!java ProblemaDelGranjero

Granjero, zorro, pollo y maiz se encuentran en el lado A.
>El granjero cruza con: 
pollo
El granjero transporta al POLLO al lado B
--LADO B--
-El GRANJERO se encuentra en el lado B
-El POLLO se encuentra en el lado B

--LADO A--
-El ZORRO se encuentra en el lado A
-El MAIZ se encuentra en el lado A


>El granjero cruza con: 
solo
El granjero cruza solo al lado A
--LADO A--
-El ZORRO se encuentra en el lado A
-El MAIZ se encuentra en el lado A
-El GRANJERO se encuentra en el lado A

--LADO B--
-El POLLO se encuentra en el lado B


>El granjero cruza con: 
zorro
El granjero transporta al ZORRO al lado B
--LADO A--
-El MAIZ se encuentra en el lado A

--LADO B--
-El ZORRO se encuentra en el lado B
-El GRANJERO se encuentra en el lado B
-El POLLO se encuentra en el lado B


>El granjero cruza con: 
pollo
El granjero transporta al POLLO al lado A
--LADO B--
-El ZORRO se encuentra en el lado B

--LADO A--
-El MAIZ se encuentra en el lado A
-El GRANJERO se encuentra en el lado A
-El POLLO se e

# **Conclusión**



Para la solución de este ejercicio creamos dos hilos, uno para cada orilla del rio, cada uno se encarga de verificar que personajes se encuentran de su lado y en el caso de quedar dos personajes incompatibles, da como resultado fallido y se finaliza la ejecución, en cambio si el hilo que corresponde al lado **B** verifica que todos se encuentran allí, se da como resultado exitoso.  
Para la sincronización utilizamos los métodos *wait()* y *notifyAll()* para que el hilo principal (quien realiza los cambios de lado según lo que requiera el usuario) despierte a los hilos de los lados **A** y **B** para verificar si los personajes que quedan sin el granjero son compatibles o si todos se encuentran en el lado **B**. Luego utilizamos un semáforo con los métodos *Semaphore.release()* y *Semaphore.acquire()* para que el hilo principal espere a que los 2 hilos analicen sus respectivos lados.
